<a href="https://colab.research.google.com/github/romashencia/hello-streamlit/blob/main/Monte_Carlo_simulations_corporate_finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
price_0 = 60
q_max = 10
q_max_e = 20
rf = 0.05
b = 40
u = 1.4
d = 1/1.4
T = 10
mc_sims = 400 #we can calibrate this number

a) Risk-neuttal probabilities

In [42]:
prob = (1 + rf - d) / (u - d)
print(round(prob, 2))

0.49


b) Calculating PV

In [36]:
# Making a random-walk model for prices
T = 10
prices = np.zeros((T + 1, mc_sims))
prices[0] = price_0

for n in range(mc_sims):
    for i in range(1, T + 1):
        if np.random.rand() < prob:
            prices[i, n] = prices[i - 1, n] * u
        else:
            prices[i, n] = prices[i - 1, n] * d


In [37]:
# Calculating PV of CF for every simulation
disc_CF = np.zeros((T + 1, mc_sims))

for n in range(mc_sims):
    for i in range(T + 1):
        if i == 0:
            disc_CF[i, n] = (price_0 - b) * q_max
        else:
            disc_CF[i, n] = (prices[i, n] - b) * q_max / (1 + rf) ** i
disc_CF


array([[200.        , 200.        , 200.        , ..., 200.        ,
        200.        , 200.        ],
       [419.04761905, 419.04761905, 419.04761905, ...,  27.21088435,
        419.04761905,  27.21088435],
       [181.40589569, 703.85487528, 703.85487528, ..., 181.40589569,
        181.40589569, -85.14970614],
       ...,
       [525.22734493, 135.36787241, 525.22734493, ..., 525.22734493,
        135.36787241, 525.22734493],
       [283.62792314,  18.41739761, 283.62792314, ..., 283.62792314,
         18.41739761, 803.44055317],
       [476.39668475, -57.63267278, 122.78265071, ..., 476.39668475,
        -57.63267278, 476.39668475]])

In [38]:
PV = np.mean(np.sum(disc_CF, axis=0))
print(f"PV of your CF's assuming no flexibility = {round(PV, 2)}")

PV of your CF's assuming no flexibility = 3433.91


**c) Optimizing our extraction depending on the
oil price**

In [39]:
disc_CF_opt = np.zeros((T + 1, mc_sims)) # new massive for cash flows
disc_CF_opt[0] = (price_0 - b) * q_max

for n in range(mc_sims):
    for i in range(1, T + 1):
        if prices[i, n] > b:
            disc_CF_opt[i, n] = (prices[i, n] - b) * q_max / (1 + rf) ** i
        else:
            disc_CF_opt[i, n] = 0


PV_opt = np.mean(np.sum(disc_CF_opt, axis=0))

print(f"PV of our CF's assuming optimization = {round(PV_opt, 2)}")

PV of our CF's assuming optimization = 3723.96


Optimized PV is higher than the previous one because now we have an option to delete negative CFs

**d) Expanding our extraction capacity for a fixed cost of I.**

We will pay *I* only when it's smaller than our PV from the previous question. Because with this option we could double our PV.
So  ***I* < 2937,5**

**e) Implementing expansion at time 0, 1 or 2**

In [40]:
#t=0
disc_CF_e_0 = np.zeros((T + 1, mc_sims))
disc_CF_e_0[0] = (price_0 - b) * q_max_e

for n in range(mc_sims):
    for i in range(1, T + 1):
        if prices[i, n] > b:
            disc_CF_e_0[i, n] = (prices[i, n] - b) * q_max_e / (1 + rf) ** i
        else:
            disc_CF_e_0[i, n] = 0

PV_opt_e_0 = np.mean(np.sum(disc_CF_e_0, axis=0))
#print(PV_opt_e_0)
print(f"We'll expand if I <= {round(PV_opt_e_0 - PV_opt, 2)}")

We'll expand if I <= 3723.96


In [26]:
#t=1
#there are 2 possible states of the world at t=1
p_u = price_0 * u
p_d = price_0 * d

prices_t1 = [p_u, p_d]

PV_exp_1 = []
PV_nonexp_1 = []

for price_t1 in prices_t1:
    PV_exp_total = np.zeros(mc_sims) #here we will keep PV's for each iteration
    PV_nonexp_total = np.zeros(mc_sims)
    for n in range(mc_sims):
        prices = np.zeros(T + 1)
        prices[0] = price_0
        prices[1] = price_t1
        for i in range(2, T + 1): # generate prices after t = 1 for each scenario taking into account risk neutral probabilities
            if np.random.rand()< prob:  #random walk
                prices[i] = prices[i - 1] * u
            else:
                prices[i] = prices[i - 1] * d
        for t in range(1, T + 1):
            PV_exp_total[n] += max(0, (prices[t-1] - b)) * q_max_e / ((1 + rf) ** (t-1)) #sum of the discounted CF's with expancion
            PV_nonexp_total[n] += max(0, (prices[t-1] - b)) * q_max / ((1 + rf) ** (t-1)) #sum of the discounted CF's without expancion
    avg_PV_exp_1 = np.mean(PV_exp_total)
    avg_PV_nonexp_1 = np.mean(PV_nonexp_total)
    PV_exp_1.append(avg_PV_exp_1)
    PV_nonexp_1.append(avg_PV_nonexp_1)

# I must be such that the current expansion costs do not exceed differences in PV's
print(f"p_u => I must be < {round(PV_exp_1[0] - PV_nonexp_1[0], 2)} for expancion")
print(f"p_d => I must be < {round(PV_exp_1[1] - PV_nonexp_1[1], 2)} for expancion")

p_u => I must be < 4670.15 for expancion
p_d => I must be < 1531.23 for expancion


In [28]:
#t=2
#there are 3 possible states of the world at t=1
p_uu = price_0 * u * u
p_ud = price_0 * u * d
p_dd = price_0 * d * d

prices_t2 = [p_uu, p_ud, p_dd]

PV_exp_2= []
PV_nonexp_2 = []

# cycles by analogy with t=1
for price_t2 in prices_t2:
    PV_exp_total = np.zeros(mc_sims)
    PV_nonexp_total = np.zeros(mc_sims)
    for n in range(mc_sims):
        prices = np.zeros(T + 1)
        prices[0] = price_t2
        for t in range(1, T + 1):
            if np.random.rand() < prob:
                prices[t] = prices[t - 1] * u
            else:
                prices[t] = prices[t - 1] * d
        for t in range(2, T + 1):
            PV_exp_total[n] += max(0, (prices[t-2] - b)) * q_max_e / ((1 + rf) ** (t-2))
            PV_nonexp_total[n] += max(0, (prices[t-2] - b)) * q_max / ((1 + rf) ** (t-2))
    avg_PV_exp_2 = np.mean(PV_exp_total)
    avg_PV_nonexp_2 = np.mean(PV_nonexp_total)
    PV_exp_2.append(avg_PV_exp_2)
    PV_nonexp_2.append(avg_PV_nonexp_2)

print(f"p_uu => I must be < {round(PV_exp_2[0] - PV_nonexp_2[0], 2)} for expancion")
print(f"p_ud => I must be < {round(PV_exp_2[1] - PV_nonexp_2[1], 2)} for expancion")
print(f"p_dd => I must be < {round(PV_exp_2[2] - PV_nonexp_2[2], 2)} for expancion")

p_uu => I must be < 8154.17 for expancion
p_ud => I must be < 2708.94 for expancion
p_dd => I must be < 681.13 for expancion


As a result, when I <= 3723.96 - choose to expand at t = 0


4670.15 <= I <= 3521.29 - choose to expand at t = 1(u)

8154.17 <= I <= 4670.15 - choose to expand at t = 2(uu)